In [1]:
import matplotlib
#matplotlib.use("nbagg")

import matplotlib.pyplot as plt
import numpy as np

In [2]:
import dlc_bci as bci
from utility import * 

## Get Data
Fetches the data from the server and saves it to a folder at the root 

In [3]:
train_input_100, train_target_100, test_input_100, test_target_100 = import100HzData()
train_input, train_target, test_input, test_target = import1000HzData()

<class 'torch.FloatTensor'> torch.Size([316, 28, 50])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 50])
<class 'torch.LongTensor'> torch.Size([100])
<class 'torch.FloatTensor'> torch.Size([316, 28, 500])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 500])
<class 'torch.LongTensor'> torch.Size([100])


In [4]:
print(train_input.shape) 
print(train_target.shape) 
print(test_input.shape) 
print(test_target.shape) 

torch.Size([316, 28, 500])
torch.Size([316])
torch.Size([100, 28, 500])
torch.Size([100])


## Visualize the data

In [5]:
#rawDataForSingleElectrodeVisualization(train_input)

# Neural Net - CNN

In [6]:
# Copy the neural network from the Neural Networks section before and modify it to take 18-channel signal (instead of 1-channel images as it was defined).

2. Define a Convolution Neural Network

In [7]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.n_filters_time1=10
        self.n_filters_time2=20
        self.n_filters_spat=25
        self.n_filters_conv=25
        self.n_input_channel=28
        self.final_conv_length=2 #!!!!!
        self.filter_time_length=5#int(10/3)
        self.conv_stride = 1 #Stride after pooling 
        self.pool_stride = 2#3 #Stride after pooling
        self.batch_norm_alpha=0.1
        self.pool_time_length=3
        self.pool_time_stride=3
        self.n_filters_2=50
        self.filter_length_2=5#int(10/3)
        self.n_filters_3=100
        self.filter_length_3=5#int(10/3)
        self.n_filters_4=200
        self.filter_length_4=5 #int(10/3)
        self.drop_prob=0.7
        self.batch_norm=True
        self.n_classes=2

        # Layer 1 - REMINDER nn.Conv2d(nbChannels, outputChannels, kxk Square convolution)
        self.a = nn.Conv2d(1, self.n_filters_time1, (1, self.filter_time_length), stride = 1) #'Time Convolution'
        self.c1 = nn.BatchNorm2d(self.n_filters_time1, momentum=self.batch_norm_alpha, affine=True, eps=1e-5)

        self.a2 = nn.Conv2d(self.n_filters_time1, self.n_filters_time2, (1, self.filter_time_length), stride = 1) #'Time Convolution'
        self.c2 = nn.BatchNorm2d(self.n_filters_time2, momentum=self.batch_norm_alpha, affine=True, eps=1e-5)

        self.b = nn.Conv2d(20, self.n_filters_spat, (self.n_input_channel, 1), stride=(self.conv_stride, 1), bias=False) #'Spatial Convolution' 
        self.c = nn.BatchNorm2d(self.n_filters_conv, momentum=self.batch_norm_alpha, affine=True, eps=1e-5)
        self.d = nn.MaxPool2d(kernel_size=(1, self.pool_time_length), stride=(1, self.pool_stride))
        
        self.e = nn.Dropout(p=self.drop_prob)


        
        '''# Layer 2
        self.e = nn.Dropout(p=self.drop_prob)
        self.f = nn.Conv2d(self.n_filters_conv, self.n_filters_2,(1, self.filter_length_2),stride=(1, self.conv_stride),
                                   bias=False)
        self.g = nn.BatchNorm2d(self.n_filters_2,momentum=self.batch_norm_alpha,affine=True,eps=1e-5)
        self.h = nn.MaxPool2d(kernel_size=(1, self.pool_time_length),stride=(1, self.pool_stride))

        # Layer 3
        self.i = nn.Dropout(p=self.drop_prob)
        self.j = nn.Conv2d(self.n_filters_2, self.n_filters_3, (1, self.filter_length_3), stride=(1, self.conv_stride),
                                   bias=False)
        self.k = nn.BatchNorm2d(self.n_filters_3,momentum=self.batch_norm_alpha,affine=True,eps=1e-5)
        self.l = nn.MaxPool2d(kernel_size=(1, self.pool_time_length), stride=(1, self.pool_stride))

        # Layer 4
        self.m =nn.Dropout(p=self.drop_prob)
        self.n =nn.Conv2d(self.n_filters_3, self.n_filters_4,
                                   (1, self.filter_length_4),
                                   stride=(1, self.conv_stride),
                                   bias=False)
        self.o =nn.BatchNorm2d(self.n_filters_4,momentum=self.batch_norm_alpha,affine=True,eps=1e-5)
        self.p =nn.MaxPool2d(kernel_size=(1, self.pool_time_length),stride=(1, self.pool_stride-1))

        #Classifier Layer
        self.q = nn.Conv2d(self.n_filters_4, self.n_classes, (1, self.final_conv_length), bias=True)
        self.r =nn.LogSoftmax(dim=-1)'''

        #self.classifier = nn.Sequential(
        self.s = nn.Linear(25*1*42, 256)
        self.t = nn.Linear(256, 128)
        self.u = nn.Linear(128, 64)
        self.v = nn.Linear(64, 2)
        
        
    def forward(self, x):
        print(x.shape)
        x = self.a(x)
        print(x.shape)
        x = self.c1(x)
        print(x.shape)
        x = self.a2(x)
        print(x.shape)
        x = self.c2(x)
        print(x.shape)
        x = self.b(x)
        print(x.shape)
        x = F.elu(self.c(x))
        print(x.shape)
        #x = self.d(x)
        #print(x.shape)
        x = self.e(x)
        print(x.shape)
        '''x = self.f(x)
        print(x.shape)
        x = self.g(x)
        print(x.shape)
        x = self.h(x)
        print('h', x.shape)
        x = self.i(x)
        print('i', x.shape)
        x = F.elu(self.j(x))
        print('j', x.shape)
        x = self.k(x)
        print('k', x.shape)
        x = self.l(x)
        print(x.shape)'''
        #x = self.m(x)
        #print(x.shape)
        #x = F.relu(self.n(x))
        #print(x.shape)
        #x = self.o(x)
        #print(x.shape)
        #x = self.p(x)
        #print('p', x.shape)
        #x = self.r(x)
        
        x = x.view(-1, 25*1*42)
        
        x = F.elu(self.s(x))
        x = self.e(x) #dropout
        x = F.elu(self.t(x)) #linear
        x = self.e(x)
        x = F.elu(self.u(x))
        x = self.e(x)
        x = F.sigmoid(self.v(x))
        print(x.shape)

        return x
net = Net()

3. Define a Loss function and optimizer
Let’s use a Classification Cross-Entropy loss and SGD with momentum.

In [8]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.4)
#criterion = nn.BCELoss()
#optimizer = optim.Adam(net.parameters())

4. Train the network
This is when things start to get interesting. We simply have to loop over our data iterator, and feed the inputs to the network and optimize.

# Data Augmentation & Data Preprocessing

You can add initial denoising, normalization and ... by setting the boolean parameters to True 

we take the 1000 hz signal and we subsample it with different time steps we downsample to 100Hz but with different initial points. By ding so we have 10 times more samples instead of having ones big weigth for the 1000hz sample



In [9]:
usePreprocessing = True

if usePreprocessing:

    preprocessed_input_train, preprocessed_input_validation, preprocessed_input_train_target, preprocessed_input_validation_target = preprocessing_train(train_input, train_target, False, False)
    preprocessed_input_test = preprocessing_test(test_input_100, False)

    print('train', preprocessed_input_train.shape)
    print('test', preprocessed_input_test.shape)
    print('validation', preprocessed_input_validation.shape)

train (2703, 28, 50)
test (100, 28, 50)
validation (144, 28, 50)


In [12]:
for epoch in range(30):  # loop over the dataset multiple times
    print ("\nEpoch ", epoch)
    running_loss = 0.0
       
    if usePreprocessing : 
        #time = np.linspace(0, 500, 50)
        #normalized_input_train = normalizedSignals(train_input, time, True, 'After Preprocessing - Simple plot of all electrodes - Train')

        inputs = torch.from_numpy(preprocessed_input_train).float()
        inputs = torch.unsqueeze(inputs,1) # make it float and insert a fake batch dimension
        #labels = train_target #y_train
        
        labels = torch.from_numpy(preprocessed_input_train_target)
        
        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()
        
        running_loss += loss.data[0]
        print("Running loss", running_loss)

        # Validation accuracy
        params = ["acc", "auc", "fmeasure"]
        
        #prediction = torch.from_numpy(normalized_input_test).float() #With normalized input 
        prediction = torch.from_numpy(preprocessed_input_validation).float() #x_train
        prediction_squeezed = torch.unsqueeze(prediction,1) # make it float and insert a fake batch dimension
        labels = torch.from_numpy(preprocessed_input_validation_target) #y_train

        # wrap them in Variable
        predictionVar, labelsVar = Variable(prediction_squeezed), Variable(labels)
        # forward + backward + optimize
        #The outputs are energies for the 10 classes. Higher the energy for a class, the more the network thinks
        #that the image is of the particular class. So, let’s get the index of the highest energy:
        outputsPred = net(predictionVar) 
        total = 0 
        correct = 0
        _, predicted = torch.max(outputsPred.data, 1) #predicted contains the predicted labels 
        total += labels.size(0)
        correct += (predicted == labels).sum()

        print('Accuracy of the network on the test singals: %d %%' % (100 * correct / total))

        
    else: 
        inputs = train_input #x_train
        inputs = torch.unsqueeze(inputs,1) # make it float and insert a fake batch dimension
        labels = train_target #y_train
        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.data[0]
        print("Running loss", running_loss)

        # Validation accuracy
        params = ["acc", "auc", "fmeasure"]


Epoch  0
torch.Size([2703, 1, 28, 50])
torch.Size([2703, 10, 28, 46])
torch.Size([2703, 10, 28, 46])
torch.Size([2703, 20, 28, 42])
torch.Size([2703, 20, 28, 42])
torch.Size([2703, 25, 1, 42])
torch.Size([2703, 25, 1, 42])
torch.Size([2703, 25, 1, 42])
torch.Size([2703, 2])
Running loss 0.69534832239151
torch.Size([144, 1, 28, 50])
torch.Size([144, 10, 28, 46])
torch.Size([144, 10, 28, 46])
torch.Size([144, 20, 28, 42])
torch.Size([144, 20, 28, 42])
torch.Size([144, 25, 1, 42])
torch.Size([144, 25, 1, 42])
torch.Size([144, 25, 1, 42])
torch.Size([144, 2])
Accuracy of the network on the test singals: 55 %

Epoch  1
torch.Size([2703, 1, 28, 50])
torch.Size([2703, 10, 28, 46])
torch.Size([2703, 10, 28, 46])
torch.Size([2703, 20, 28, 42])
torch.Size([2703, 20, 28, 42])
torch.Size([2703, 25, 1, 42])
torch.Size([2703, 25, 1, 42])
torch.Size([2703, 25, 1, 42])
torch.Size([2703, 2])
Running loss 0.6980694532394409
torch.Size([144, 1, 28, 50])
torch.Size([144, 10, 28, 46])
torch.Size([144, 10,

Running loss 0.6989833116531372
torch.Size([144, 1, 28, 50])
torch.Size([144, 10, 28, 46])
torch.Size([144, 10, 28, 46])
torch.Size([144, 20, 28, 42])
torch.Size([144, 20, 28, 42])
torch.Size([144, 25, 1, 42])
torch.Size([144, 25, 1, 42])
torch.Size([144, 25, 1, 42])
torch.Size([144, 2])
Accuracy of the network on the test singals: 51 %

Epoch  14
torch.Size([2703, 1, 28, 50])
torch.Size([2703, 10, 28, 46])
torch.Size([2703, 10, 28, 46])
torch.Size([2703, 20, 28, 42])
torch.Size([2703, 20, 28, 42])
torch.Size([2703, 25, 1, 42])
torch.Size([2703, 25, 1, 42])
torch.Size([2703, 25, 1, 42])
torch.Size([2703, 2])
Running loss 0.6951611042022705
torch.Size([144, 1, 28, 50])
torch.Size([144, 10, 28, 46])
torch.Size([144, 10, 28, 46])
torch.Size([144, 20, 28, 42])
torch.Size([144, 20, 28, 42])
torch.Size([144, 25, 1, 42])
torch.Size([144, 25, 1, 42])
torch.Size([144, 25, 1, 42])
torch.Size([144, 2])
Accuracy of the network on the test singals: 44 %

Epoch  15
torch.Size([2703, 1, 28, 50])
torc

torch.Size([2703, 10, 28, 46])
torch.Size([2703, 10, 28, 46])
torch.Size([2703, 20, 28, 42])
torch.Size([2703, 20, 28, 42])
torch.Size([2703, 25, 1, 42])
torch.Size([2703, 25, 1, 42])
torch.Size([2703, 25, 1, 42])
torch.Size([2703, 2])
Running loss 0.6961077451705933
torch.Size([144, 1, 28, 50])
torch.Size([144, 10, 28, 46])
torch.Size([144, 10, 28, 46])
torch.Size([144, 20, 28, 42])
torch.Size([144, 20, 28, 42])
torch.Size([144, 25, 1, 42])
torch.Size([144, 25, 1, 42])
torch.Size([144, 25, 1, 42])
torch.Size([144, 2])
Accuracy of the network on the test singals: 48 %

Epoch  28
torch.Size([2703, 1, 28, 50])
torch.Size([2703, 10, 28, 46])
torch.Size([2703, 10, 28, 46])
torch.Size([2703, 20, 28, 42])
torch.Size([2703, 20, 28, 42])
torch.Size([2703, 25, 1, 42])
torch.Size([2703, 25, 1, 42])
torch.Size([2703, 25, 1, 42])
torch.Size([2703, 2])
Running loss 0.6945803761482239
torch.Size([144, 1, 28, 50])
torch.Size([144, 10, 28, 46])
torch.Size([144, 10, 28, 46])
torch.Size([144, 20, 28, 42

5. Test the network on the test data
We have trained the network for 2 passes over the training dataset. But we need to check if the network has learnt anything at all.

We will check this by predicting the class label that the neural network outputs, and checking it against the ground-truth. If the prediction is correct, we add the sample to the list of correct predictions.

Okay, first step. Let us display an image from the test set to get familiar.

# Test Dataset - Prediction 

In [13]:
correct = 0
total = 0

if usePreprocessing: 
    #normalized_input_test = normalizedSignals(test_input, time, True, 'After Preprocessing - Simple plot of all electrodes - Test')   
    #prediction = torch.from_numpy(normalized_input_test).float() #With normalized input 
    #prediction = test_input_100 #x_train    
    preprocessed = torch.from_numpy(preprocessing_test(test_input_100)).float()
    prediction_squeezed = torch.unsqueeze(preprocessed,1) # make it float and insert a fake batch dimension
    labels = test_target #y_train

    # wrap them in Variable
    predictionVar, labelsVar = Variable(prediction_squeezed), Variable(labels)

    # forward + backward + optimize
    #The outputs are energies for the 10 classes. Higher the energy for a class, the more the network thinks
    #that the image is of the particular class. So, let’s get the index of the highest energy:
    outputsPred = net(predictionVar) 

    _, predicted = torch.max(outputsPred.data, 1) #predicted contains the predicted labels 
    total += labels.size(0)
    correct += (predicted == labels).sum()

    print('Accuracy of the network on the test singals: %d %%' % (100 * correct / total))
    
else: 
    #prediction = torch.from_numpy(normalized_input_test).float() #With normalized input 
    prediction = test_input_100 #x_train
    prediction_squeezed = torch.unsqueeze(prediction,1) # make it float and insert a fake batch dimension
    labels = test_target #y_train

    # wrap them in Variable
    predictionVar, labelsVar = Variable(prediction_squeezed), Variable(labels)

    # forward + backward + optimize
    #The outputs are energies for the 10 classes. Higher the energy for a class, the more the network thinks
    #that the image is of the particular class. So, let’s get the index of the highest energy:
    outputsPred = net(predictionVar) 

    _, predicted = torch.max(outputsPred.data, 1) #predicted contains the predicted labels 
    total += labels.size(0)
    correct += (predicted == labels).sum()

    print('Accuracy of the network on the test singals: %d %%' % (100 * correct / total))
    

torch.Size([100, 1, 28, 50])
torch.Size([100, 10, 28, 46])
torch.Size([100, 10, 28, 46])
torch.Size([100, 20, 28, 42])
torch.Size([100, 20, 28, 42])
torch.Size([100, 25, 1, 42])
torch.Size([100, 25, 1, 42])
torch.Size([100, 25, 1, 42])
torch.Size([100, 2])
Accuracy of the network on the test singals: 38 %
